In [99]:
from arcticdb import Arctic
ac = Arctic("lmdb:///Users/baeyeongmin/arcticdb")
DELTA_DB_PATH='/Users/baeyeongmin/deltadb'
date='20230915'

In [100]:
import os
from glob import glob
logs_path = os.path.join(DELTA_DB_PATH, date, "ticks", "*.csv")
paths = glob(logs_path)

In [101]:
import pandas as pd
import polars as pl

In [102]:
ORDERBOOK_COLS = [
 'offerrem2',
 'offerho4',
 'bidho5',
 'offerho3',
 'offerrem3',
 'bidho4',
 'offerrem4',
 'offerho6',
 'bidho7',
 'offerho5',
 'offerrem5',
 'bidho6',
 'offerho8',
 'bidho9',
 'offerho7',
 'bidho8',
 'offerrem1',
 'offerho9',
 'offerrem6',
 'offerrem7',
 'offerrem8',
 'offerrem9',
 'bidrem3',
 'bidrem4',
 'bidrem1',
 'bidrem2',
 'bidrem9',
 'bidho1',
 'bidrem7',
 'bidrem8',
 'hotime',
 'offerho2',
 'bidho3',
 'bidrem5',
 'offerho1',
 'bidho2',
 'bidrem6',
 'bidrem10',
 'bidho10',
 'shcode',
 'offerho10',
 'offerrem10',
 'local_timestamp'
]

In [103]:
# df = pd.read_csv(paths[0], usecols=ORDERBOOK_COLS, index_col=['local_timestamp'])

In [104]:
df = pl.read_csv(paths[0], try_parse_dates=True, infer_schema_length=10_000)

In [105]:
df = df[ORDERBOOK_COLS]

In [107]:
df = df.rename({
    col: col.replace("bidho", "bpx").replace("bidrem", "bqty").replace("offerho", "apx").replace("offerrem", "aqty").replace('shcode', 'symbol').replace('hotime', 'exchange_timestamp')
    for col in df.columns
})
df = df.select(sorted(df.columns))

In [87]:
from datetime import datetime

In [92]:
def combine_date_time(date_obj, time_str):
    time_str = f"{time_str:06}"
    hour = int(time_str[0:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:6])
    return datetime(date_obj.year, date_obj.month, date_obj.day, hour, minute, second)


In [95]:
df['exchange_timestamp'] = df['exchange_timestamp'].apply(lambda x: combine_date_time(date_obj=today, time_str=x))

In [135]:
if 'ebest.orderbook3' not in ac.list_libraries():
    ac.create_library('ebest.orderbook3')
lib = ac.get_library('ebest.orderbook3')

In [138]:
from tqdm import tqdm

In [139]:
symbols = df.group_by('symbol').agg().to_dicts()
for symbol in tqdm(symbols):
    orderbook_np = df.filter(df['symbol'] == symbol['symbol']).to_numpy(structured=True)
    orderbook_df = pd.DataFrame(orderbook_np, columns=df.columns)
    orderbook_df.index = pd.to_datetime(orderbook_df['local_timestamp'], unit='s')
    orderbook_df.drop(columns=['local_timestamp'], inplace=True)
    orderbook_df['exchange_timestamp'] = orderbook_df['exchange_timestamp'].apply(lambda x: combine_date_time(date_obj=today, time_str=x))
    orderbook_df.sort_index(inplace=True)
    lib.write(symbol['symbol'], orderbook_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [00:25<00:00, 81.49it/s]
